* 25 % carga y limpieza
    * Carga con Pandas: pd.read_csv

    * Limpieza de valores error: 
        * hay columnas que tienen un valor '?', por tanto se deben reemplazar por un valor nan.

    * Cambio de tipo de dato: .astype() a numéricos y textos, por ejemplo a categorical y carat a float32 o float16

    * Limpieza de nulos (limpiar valores NaN):
        * Nulos en columnas continuas: mediana, media
        * Nulos en columnas categóricas: moda, un valor fijo

    * Encoding: texto a numérico
        * Uso de la función get_dummies() para encoding one_hot
        * Uso de map para encoding ordinal para la columna cut: 1, 2, 3, 4

* 25 % transformaciones:
    * Uso de función apply:
        * crear una columna price_iva a partir de la columna price que muestre el precio + IVA (21%)

    * Crear una nueva columna volumen combinando: x * y * z

    * Ordenar por dos columnas con sort_values():
        * tipo de corte (cut) y precio (price)

    * Agrupaciones con groupby y visualizarla
        * Agrupar por las 3 que hay de tipo categórico calculando la media, max, min por ejemplo de alguna de las numéricas: price, carat, depth

20 % distribuciones: 

* Outliers: Visualización Q1 y Q3 y calcular límites tukey y filtrar. Sobre la columna precio.

* asimetría, curtosis y transformar datos con logaritmo o raíz cuadrada

* Discretizar la columna precio por barato, medio, caro usando la función pd.cut

30 % visualizaciones:

* Seaborn EDAS:
    * univariantes:
        * histogramas y curvas de densidad
        * boxplot
        * countplot
    * bivariantes y multivariantes
        * scatterplot con hue, con size, con style
        * Calcular correlación con Pandas y mostrarla con seaborn
        * Hacer la correlación en un gráfico de barras para la columna 'price'
        * Crear una pivot table usando como index y columns algunas variables categóricas y como values usar el price y visualizarla con heatmap de seaborn
    * Combinarlas con:
        * hue, style, size, row, col usando un relplot
        * filtro


### Librerías a utilizar

In [87]:
import numpy as np
import pandas as pd
import seaborn as sns

# Carga y Limpieza
### Carga csv con pandas

In [ ]:
df = pd.read_csv("diamonds.csv")
df.head(2)

In [ ]:
df.info()


### Limpieza de valores error

In [105]:
df = df.replace('?', np.nan)

### Cambio de tipo de datos

In [107]:
df["carat"] = df["carat"].astype("float32")
df["price"] = df["price"].astype("float")
df["x"] = df["x"].astype("float32")
df["cut"] = df["cut"].astype("category")

In [ ]:
df.info()

### Limpieza NaN

In [ ]:
df.count()

In [ ]:
df.isna().sum()

In [ ]:
df["carat"] = df["carat"].fillna(df["carat"].mean())
df["cut"] = df["cut"].ffill()
df["color"] = df["color"].fillna(method="ffill")
df["clarity"] = df["clarity"].fillna(method="ffill")
df["table"] = df["table"].fillna(df["table"].mean())
df["price"] = df["price"].fillna(df["price"].mean())
df["price"] = df["price"].astype("int32")
df["x"] = df["x"].fillna(method="ffill")
df["y"] = df["y"].fillna(df["y"].mean())
df["z"] = df["z"].fillna(df["z"].mean())

In [ ]:
df.info()


## Encoding

* Get_dummies()

In [50]:
#

* Encodig Ordinal

In [51]:
#

### Creacion columnas

* price_iva

In [52]:
#

* volumen ( x, y, z)

In [53]:
#

### Orden de columnas


* Tipo de corte

* Precio

### Agrupaciones group_by()

* carac

In [54]:
#

* price

In [55]:
#

* depth

In [56]:
#

## Seaborn EDAS
### Univariantes

* Histograma y curva de densidad

In [57]:
#

* Boxplot

In [58]:
#

* Countplot

In [59]:
#

### Bivariantes y Multivariantes
* Scatterplot

In [60]:
# con hue


In [61]:
# con style


In [62]:
# con size

#### Correlacion
* Calculo con pandas

In [63]:
#

* Mostrar con Seaborn

In [64]:
#

* Combinarlas con filtros hue, style

In [65]:
#

## Outliers
* Visualizacion Q1 y Q3 

In [66]:
#

* Limites Tukey y filtro

In [67]:
#

## Transformacion de datos
* Asimetria

* Curtosis

* Transformacion logaritmo

* Transformacion raiz cuadrada

## Discretizar la columna precio
* barato, medio, caro